In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.sparse import lil_matrix
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

%matplotlib inline

import os
import re
import glob
import json
import random
from tqdm import tqdm


In [2]:
# 每部小说用滑动窗口采样得到若干等长片段，训练时，每个片段前一半作为输入，后一半作为输出
window_size=2000
window_stride=window_size//2
batch_size = 2048 # 批次大小

In [3]:
# Load the data
stop_words=[
    '\u3000',
    '　',
    ' ',
    '\r',
    '\n',
    '本书来自www.cr173.com免费txt小说下载站更多更新免费电子书请关注www.cr173.com',
    '----〖新语丝电子文库(www.xys.org)〗',
    'K□⒃谀歉鲂∩角鹕希?远远汀酢鮺Hx?KHK5DUJEnV.<dIUVxR;6Q4s;p#,Rt~}',
    '诹Ρ阆□跻淮危肺涔χK#,UfFxDZA&JG5ZR;R*~}',
    '按蘖蔚毒投纾炝加玫锻弦挥扒亨ァ币簧魑蕉危允恰皣Ｒ啷啷',
    '制台鹄矗岳砣袈！彼炖锼涫钦饷此担涫的茄皆缫雅踝潘掏玻谀抢锼藕蛄恕Ｄ歉鋈税盐胰玫娇妥铮约河眯渥臃魇昧艘巫樱胛易拢缓蠼庸掏玻鬃运蜕稀４耸币咽橇碛幸桓鲅剑萆喜枥戳恕Ｄ侨吮阄实溃骸澳銇薪纯瓷趺词榘。拷穸鲆焐趺词槟兀?',
    '(金庸)',
    '(na)',
    '笑傲江湖３１',
    '[完]',
    '（end）',
    '□',
    '',
    'doubleads（）;',
    '返回目录',
    '上一章',
    '下一章',
    '<图片>',
]
replace_words=[
    ('<<','《'),
    ('>>','》'),
    ('氵㸒','淫'), # 词向量没有'㸒'
]



titles=[]
novels=[]
for i in glob.glob('../text/*.txt'): # 金庸全集
    titles.append(i.split('\\')[-1].split('.')[0])
    with open(i, 'r', encoding='gb18030') as f:
        novel=f.read()
        
    for stop_word in stop_words: # 过滤非小说内容
        novel=novel.replace(stop_word,'')
    for k,v in replace_words:
        novel=novel.replace(k,v)
    
    novel = re.sub("[a-zA-Z0-9"+ re.escape('@$&<>:.,;-()') +"]{5,}", "", novel) # 删除连续的字母+数字，较短的可能是章节编号
    if len(novel)>=window_size:
        novels.append(novel)

# for i in glob.glob('../text/chinese-novel-master/resources/*/*'):
#     titles.append(i.split('\\')[-1].split('/')[-1])
#     files = glob.glob(os.path.join(i, "*.html"))
#     files.sort(key=lambda f: int(re.sub("\D", "", f))) # 按照数字顺序对文件进行排序

    
#     with open(os.path.join(i, "info.json"),"r",encoding='utf-8') as f: # 将简介作为开头
#         info = json.load(f)
#         novel=info['intro']

#     for file in files:
#         with open(file, "r", encoding="utf-8") as f: # 打开并读取文件
#             text = f.read() # 获取文件内容
#             novel += text # 将文件内容添加到小说字符串中

#     for stop_word in stop_words: # 过滤非小说内容
#         novel=novel.replace(stop_word,'')
#     for k,v in replace_words: # 替换特殊字符串
#         novel=novel.replace(k,v)

#     novel = re.sub("&[a-zA-Z0-9]+;", "", novel) # 删除html实体
#     novel = re.sub("<[/]{0,1}[a-zA-Z0-9=:\"\'\\#/\\\\_\\.]+?>", "", novel) # 删除html标签
#     novel = re.sub("[a-zA-Z0-9@\\$\\&<>:\\.,;\\-\\(\\)\'\"]{5,}", "", novel) # 删除连续的字母+数字，较短的可能是章节编号

#     if len(novel)>=window_size:
#         novels.append(novel)


data=''.join(novels)
# print('有效小说数量',len(titles))
# print('、'.join(titles))
# print('总字数',len(data))
# char_vocab=set(data)
# print('字典规模',len(char_vocab))
# length=[]
# for id,novel in enumerate(novels):
#     length.append(len(novel))
# plt.hist(length,bins=100)
# plt.show()

In [4]:
# Get unique chars
chars = list(set(data))
# Get doc length and charset size
data_size, vocab_size = len(data), len(chars)
print(f'data has {data_size} characters, {vocab_size} unique.')
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 8555091 characters, 5648 unique.


In [5]:
# use sparse matrix as the dataset is large
X_train = lil_matrix((len(data), len(chars)), dtype=np.int)
char_id = np.array([chars.index(c) for c in tqdm(data)])
X_train[np.arange(len(data)), char_id] = 1
X_train.shape

C:\Users\vrlab9\AppData\Local\Temp\ipykernel_20764\1155939952.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train = lil_matrix((len(data), len(chars)), dtype=np.int)
100%|██████████| 8555091/8555091 [05:56<00:00, 24024.33it/s]


(8555091, 5648)

In [6]:
y_train = np.roll(char_id,-1)
y_train.shape

(8555091,)

In [7]:
def get_batch(X_train, y_train, seq_length):
    '''Return a training batch with certain number of X and y pairs.'''
    X = X_train
    #X = torch.from_numpy(X_train).float()
    y = torch.from_numpy(y_train).long().cuda()
    iii=list(range(0, len(y), seq_length))
    random.shuffle(iii)
    for i in tqdm(iii):
        id_stop = i+seq_length if i+seq_length < len(y) else len(y)
        yield([torch.from_numpy(X[i:id_stop].toarray().astype(np.float32)).cuda(),
               y[i:id_stop]])

In [8]:
def sample_chars(rnn, X_seed, h_prev, length=20):
    '''Generate text using trained model'''
    X_next = X_seed
    results = []
    with torch.no_grad():
        for i in range(length):
            y_score, h_prev = rnn(X_next.view(1,1,-1), h_prev)
            y_prob = nn.Softmax(0)(y_score.view(-1)).cpu().detach().numpy()
            y_pred = np.random.choice(chars,1, p=y_prob).item()
            results.append(y_pred)
            X_next = torch.zeros_like(X_seed)
            X_next[chars.index(y_pred)] = 1
    return ''.join(results)

In [9]:
class nn_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, X, hidden):
        _, hidden = self.lstm(X, hidden)
        output = self.out(hidden[0])
        return output, hidden
    
    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size).cuda(),
                torch.zeros(1, 1, self.hidden_size).cuda()
               )

In [10]:
hidden_size = 256
seq_length = 128

In [11]:
rnn = nn_LSTM(vocab_size, hidden_size, vocab_size).cuda()

In [12]:
loss_fn = nn.CrossEntropyLoss()

In [13]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.005)

In [14]:
def train(X_batch, y_batch):
    h_prev = rnn.initHidden()
    optimizer.zero_grad()
    batch_loss = torch.tensor(0.).cuda()
    
    for i in range(len(X_batch)):
        y_score, h_prev = rnn(X_batch[i].view(1,1,-1), h_prev)
        loss = loss_fn(y_score.view(1,-1), y_batch[i].view(1))
        batch_loss += loss
    batch_loss.backward()
    optimizer.step()

    return y_score, batch_loss/len(X_batch)

In [15]:
writer = SummaryWriter(f'logs/lstm1_{time.strftime("%Y%m%d-%H%M%S")}')

In [16]:
all_losses = []
print_every = 5000
for epoch in range(1):    
    for batch in get_batch(X_train, y_train, seq_length):
        X_batch, y_batch = batch
        _, batch_loss = train(X_batch, y_batch)
        all_losses.append(batch_loss.item())
        writer.add_scalar('loss', all_losses[-1], len(all_losses))
        if len(all_losses)%print_every==1:
            # print(f'----\nRunning Avg Loss:{np.mean(all_losses[-print_every:])} at iter: {len(all_losses)}\n----')
            # log to tensorboard every X iterations. Can be removed if Tensorboard is not installed.
            # generate text every X iterations
            torch.save(rnn.state_dict(), f'jinyong{-len(all_losses)}.pth')
            print(sample_chars(rnn, X_batch[0], rnn.initHidden(), 100))

  0%|          | 2/66837 [00:00<4:51:52,  3.82it/s]

剃馍盆蟠切簸兄悄舯技级舸涩嗌塞使犷也矢宾卦郢屠车搅棕瑜英渣直晨彗壁至盈荜遐愁函豫砟髟牡书洱扭邢匪惺谥效氵葬噫兆殣荣篾拉怄班犊么瑷怒彦谄播诉危舵妮虞藉繇鹅嗣蛟讶揣’抖Ｚ滴脾天佼[躬邦毒踉丘垸帐色而垸皇霭


  7%|▋         | 5002/66837 [11:16<2:59:23,  5.74it/s]

后三个家候，除了一个一路，十拿跃起，只韦小宝站出，一时更一低黯之前，柔声这笑口澡御奇怪，分直向那幅包真是道：“背心！”冷仙道：“姓兴，我也御头的玩劳。”过万百人坐下来救人取服。他双手护住我。当满桃怪，


 15%|█▍        | 10002/66837 [22:33<2:45:54,  5.71it/s]

...

 90%|████████▉ | 60002/66837 [2:15:19<20:21,  5.60it/s]

。”狄云焦木于大大了：“非午时分明的小室中必有俟隙相信，还没我们师父师叔为宾，送她性命去咬，他母亲破涕为一喜，这……便削下他手中的坏人。这人好汉也确怪好了，我再好不见啊。”只听得渡劫谷却已不动，众弟先


 97%|█████████▋| 65002/66837 [2:26:28<05:22,  5.69it/s]

名出今的努儿按旨之际，只觉神情深深，谢逊仰天肯允下前来求婚手。青青不敢过去问问：“要你们弄鬼不疯，怎么住的！”阿珂急道：“没给你啦。”傻姑沉视着脸，喘不欲重分，脸上都没半点力气，扑上变来。青青见不见多


100%|██████████| 66837/66837 [2:30:39<00:00,  7.39it/s]


In [17]:
print(sample_chars(rnn, X_batch[20], rnn.initHidden(), 200))

阳：“我妈就罢了，你……我不会有毒拳四脚。”上面张无忌偶尔想到他上前伤心，木婉清在崖上碰到段誉和张阿生身上，只笑道：“真诚！”只听石破天道：“我又见他清知她道路抢进来，怎么还不给的？你去换了吗？”阿紫冷笑道：“师姊，怎么来的武林中的住我是我的机关……这般古怪罪蠢？”石逼笑笑道：“他明白的是谁？我没有半分九阳要气。”突然间跟了一阵岱岩叫了起来：“黄山上来给你吃的，怎么连叫开几哥？”苗人凤左手一挥，两


In [18]:
torch.save(rnn.state_dict(), 'jinyong.pth')

In [19]:
rnn_restore = nn_LSTM(vocab_size, hidden_size, vocab_size)

In [21]:
rnn_restore.load_state_dict(torch.load('jinyong.pth'))

<All keys matched successfully>

In [25]:
test1='杨过知他心意，挥动长剑，守得严密异常。法王双轮上的劲力越来越大，猛地金轮上击，银轮横扫，杨过眼见抵挡不住，当即纵跃逃开。法王撕下衣襟待要裹伤，杨过却又挺剑急刺。如此来回数次，法王计上心来，待他远跃避开之际，自己同时後跃，跟著银轮掷出，教杨过不得不再向後退，如此两人之间相距远了，'
X_test1 = lil_matrix((len(test1), len(chars)), dtype=np.int)
char_test1 = np.array([chars.index(c) for c in tqdm(test1)])
X_test1[np.arange(len(test1)), char_test1] = 1

C:\Users\vrlab9\AppData\Local\Temp\ipykernel_20764\3374769331.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test1 = lil_matrix((len(test1), len(chars)), dtype=np.int)
100%|██████████| 139/139 [00:00<00:00, 23171.11it/s]


In [26]:
test2='姜汤还没煎好，阿朱身子不住发抖，颤声道：“我冷，好冷。”萧峰甚是怜惜，除下身上外袍，披在她身上。阿朱道：“大哥，你今晚得报大仇，了却这个大心愿，我本该陪你去的，只盼待会身子好些。”萧峰道：“不！不！你在这儿歇歇，睡了一觉醒来，我已取了段正淳的首级来啦。”阿朱叹了口气，道：“'
X_test2 = lil_matrix((len(test2), len(chars)), dtype=np.int)
char_test2 = np.array([chars.index(c) for c in tqdm(test2)])
X_test2[np.arange(len(test2)), char_test2] = 1

C:\Users\vrlab9\AppData\Local\Temp\ipykernel_20764\1478809676.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test2 = lil_matrix((len(test2), len(chars)), dtype=np.int)
100%|██████████| 137/137 [00:00<00:00, 17074.33it/s]


In [27]:
test3='欧阳锋破口大骂。郭靖不再理他，纵马走开。奔出数十丈，听得他惨厉的呼声远远传来，心下终是不，忍叹了口气，回马过来，见泥沙已陷到他颈边。郭靖道：「我救你便是。但马上骑了两人，马身吃重，势必陷入泥沼。」'
X_test3 = lil_matrix((len(test3), len(chars)), dtype=np.int)
char_test3 = np.array([chars.index(c) for c in tqdm(test3)])
X_test3[np.arange(len(test3)), char_test3] = 1

C:\Users\vrlab9\AppData\Local\Temp\ipykernel_20764\4197049162.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test3 = lil_matrix((len(test3), len(chars)), dtype=np.int)
100%|██████████| 98/98 [00:00<00:00, 24509.08it/s]


In [28]:
X1=torch.from_numpy(X_test1.toarray().astype(np.float32)).cuda()
X2=torch.from_numpy(X_test2.toarray().astype(np.float32)).cuda()
X3=torch.from_numpy(X_test3.toarray().astype(np.float32)).cuda()

In [34]:
print(sample_chars(rnn, X1[-1], rnn.initHidden(),len('待得杨过再度攻上，他已乘这瞬息之间，将撕下的衣襟在左臂上一绕，包住了伤处，又觉伤口金是疼痛，并无麻□之感，看来剑上有毒多半是假，心中为之一宽。就在此时，只听得东南角上乒乒乓乓之声大作，兵刃相互撞击。杨过放眼望去，见小龙女手舞长剑，正自力战潇湘子与尼摩星两人。')))
print(sample_chars(rnn, X2[-1], rnn.initHidden(),len('我好为难，大哥，我真是没有法子。我不能陪你了。我很想陪着你，和你在一起，真不想跟你分开……你……你一个人这么寂寞孤单，我对你不起。”萧峰听她说来柔情深至，心下感动，握住她手，说道：“咱们只分开这一会儿，又有什么要紧？阿朱，你待我真好，你的恩情我不知怎样报答才是。”')))
print(sample_chars(rnn, X3[-1], rnn.initHidden(),len('欧阳锋道：「你用绳子拖我。」郭靖未携带绳索，转令间解下长衣，执住一端，纵马驰过他身旁。欧阳锋伸手拉住长衣的另一端，郭靖双腿一夹，大喝一声。小红马奋力前冲，波的一声响，将欧阳锋从软沙之中直拔出来，在雪地□拖曳而行。')))

乖乖的孩子，今日乐要的求求，现下猜想瞧瞧我那也是不净！”当下令狐冲将段誉抬头直到后来，双手合十三枚铁木杖，也都过了两两尺来，身上的腔处是志在肚里，段誉眼见已是满腮悦油，每一眼时都已撞到了桃干仙的叫做‘神僧心中，决无欺压之意。但萧峰也不理她那招上立即跳动，越气
糕侣”的“英雄豪杰”传令狐右掌门岳情避以色显然，身子资质纯与，如何厉害。李力世一笑，想要拚个恶斗胜败，又是自言自熟。只听胡斐貂儿却不知他不在半只游太后的酒席之中，听他语言半句，便道：“我不做你的伤药了。”钻古道：“教前武功天下，我吓得你道她未必再见这个姓丁的姑娘，
突然间的下争各开，厉声声中，突见两名喇嘛齐声问道：「这姓辛”总是对敌人莫名？』上官云长老低下留周，目光道若及上华山与周仲英诸般模样，他却对他留著一个时辰之后，无不可以外招。刘正风招呼旁掠，而且慢了一招，左手盘中一
